In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("atifaliak/youtube-comments-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/youtube-comments-dataset


In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv(path + "/YoutubeCommentsDataSet.csv")
df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [ ]:
df.shape

(18408, 2)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')   # for better lemmatization results
nltk.download('punkt')     # for tokenizing sentences
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')  # for POS tagging
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

In [ ]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [ ]:
import string
def remove_punc(text):
  punc = string.punctuation
  for char in punc:
    text = text.replace(char,'')
  return text

In [ ]:
import re
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    new_text = []
    for word in text.split():
        if word.lower() not in stopwords.words('english'):
            new_text.append(word)
    return ' '.join(new_text)

In [ ]:
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# POS tag mapping for WordNet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun

# Lemmatize with correct POS
def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(sentence)
    tagged_tokens = pos_tag(tokens)

    lemmatized = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag))
        for word, pos_tag in tagged_tokens
    ]
    return ' '.join(lemmatized)

In [ ]:
df['Sentiment'] = df['Sentiment'].replace(
    {
        "positive" : 1,
        "negative" : -1,
        "neutral"  : 0
    }
)

/tmp/ipython-input-14-3220413170.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sentiment'] = df['Sentiment'].replace(


In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['Comment'] = df['Comment'].apply(remove_html_tags)
df['Comment'] = df['Comment'].apply(remove_url)
df['Comment'] = df['Comment'].apply(remove_punc)
df['Comment'] = df['Comment'].apply(remove_emoji)
df['Comment'] = df['Comment'].apply(remove_stopwords)

In [ ]:
df['Comment'] = df['Comment'].apply(lemmatize_sentence)

In [ ]:
df

,Comment,Sentiment
0,let forget apple pay 2014 require brand new ip...,0
1,nz 50 retailer don ’ t even contactless credit...,-1
2,forever acknowledge channel help lesson ideas ...,1
3,whenever go place doesn ’ t take apple pay doe...,-1
4,apple pay convenient secure easy use used kore...,1
...,...,...
17869,really like point engineering toolbox think th...,1
17870,i ’ ve start explore field really good reminde...,1
17871,excelente video con una pregunta filosófica pr...,0
17872,hey daniel discover channel couple day ago im ...,1


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Comment'], df['Sentiment'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Linear Models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron

# Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import CategoricalNB

# Support Vector Machines
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC

# Decision Trees & Ensembles
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid

# Discriminant Analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Neural Networks
from sklearn.neural_network import MLPClassifier

# XGBoost
from xgboost import XGBClassifier

# LightGBM
from lightgbm import LGBMClassifier

# CatBoost (handles categorical data automatically)
from catboost import CatBoostClassifier

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix
)
from sklearn.preprocessing import FunctionTransformer

In [ ]:
models = {
    # Discriminant Analysis
    # "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),
    # "Quadratic Discriminant Analysis": QuadraticDiscriminantAnalysis(),

    # # Neural Network
    # "MLP Classifier": MLPClassifier(),

    # Gradient Boosting Alternatives (External)
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=1),

    # Nearest Neighbors
    "K-Nearest Neighbors": KNeighborsClassifier(),
    # "Radius Neighbors": RadiusNeighborsClassifier(),
    "Nearest Centroid": NearestCentroid(),

    # Linear Models
    "Logistic Regression": LogisticRegression(),
    "Ridge Classifier": RidgeClassifier(),
    "SGD Classifier": SGDClassifier(),
    "Passive Aggressive Classifier": PassiveAggressiveClassifier(),
    "Perceptron": Perceptron(),

    # Naive Bayes
    "Gaussian NB": GaussianNB(),
    "Multinomial NB": MultinomialNB(),
    "Bernoulli NB": BernoulliNB(),
    "Complement NB": ComplementNB(),

    # # SVM
    # "SVC (RBF Kernel)": SVC(),
    # "Linear SVC": LinearSVC(),
    # "Nu SVC": NuSVC(),

    # Tree-Based
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "HistGradient Boosting": HistGradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Bagging": BaggingClassifier(),


}


In [ ]:
vectorizers = {
    "Count Vectorizer": CountVectorizer(max_features=5000),
    "TF-IDF Vectorizer": TfidfVectorizer(max_features=5000)
}

In [ ]:
import dagshub
import mlflow
from urllib.parse import urlparse
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
import pandas as pd

dagshub.init(repo_owner='farjhanahmed', repo_name='Demo-Exp', mlflow=True)

def evaluate_model(models, vectorizers, X_train, X_test, y_train, y_test):
    models_name = []
    vectorizers_name = []
    accuracy_metrix = []

    mlflow.set_tracking_uri('https://dagshub.com/farjhanahmed/Demo-Exp.mlflow')
    mlflow.set_experiment('Demo-Exp')

    for model_name, model in models.items():
        for vectorizer_name, vectorizer in vectorizers.items():
            run_name = f"{model_name}_{vectorizer_name}"
            with mlflow.start_run(run_name=run_name):
                print({'model_name': model_name, 'vectorizer_name': vectorizer_name})

                pipeline = make_pipeline(
                    vectorizer,
                    FunctionTransformer(lambda x: x.toarray(), accept_sparse=True),
                    model
                )
                pipeline.fit(X_train, y_train)
                y_pred = pipeline.predict(X_test)

                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='micro')
                recall = recall_score(y_test, y_pred, average='micro')
                f1 = f1_score(y_test, y_pred, average='micro')
                cm = confusion_matrix(y_test, y_pred)[0][0]

                mlflow.log_param("model_name", model_name)
                mlflow.log_param("vectorizer_name", vectorizer_name)
                mlflow.log_metrics({
                    "accuracy": accuracy,
                    "precision": precision,
                    "recall": recall,
                    "f1_score": f1,
                    "confusion_matrix_value": cm
                })

                models_name.append(model_name)
                vectorizers_name.append(vectorizer_name)
                accuracy_metrix.append(accuracy)

    result = pd.DataFrame(zip(models_name, vectorizers_name, accuracy_metrix),
                          columns=['model_name', 'vectorizer_name', 'accuracy'])
    result = result.sort_values(by='accuracy', ascending=False)
    return result


Accessing as farjhanahmed

Initialized MLflow to track repo "farjhanahmed/Demo-Exp"

Repository farjhanahmed/Demo-Exp initialized!

In [ ]:
result = evaluate_model(models, vectorizers, X_train, X_test, y_train, y_test)

{'model_name': 'LightGBM', 'vectorizer_name': 'Count Vectorizer'}


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.156940 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6011
[LightGBM] [Info] Number of data points in the train set: 14299, number of used features: 1763
[LightGBM] [Info] Start training from score -2.039076
[LightGBM] [Info] Start training from score -1.380089
[LightGBM] [Info] Start training from score -0.480790


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


🏃 View run LightGBM_Count Vectorizer at: https://dagshub.com/farjhanahmed/Demo-Exp.mlflow/#/experiments/1/runs/2dfab1abdc004976b5c76d8005c1f919
🧪 View experiment at: https://dagshub.com/farjhanahmed/Demo-Exp.mlflow/#/experiments/1
{'model_name': 'LightGBM', 'vectorizer_name': 'TF-IDF Vectorizer'}


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.164317 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 53414
[LightGBM] [Info] Number of data points in the train set: 14299, number of used features: 1763
[LightGBM] [Info] Start training from score -2.039076
[LightGBM] [Info] Start training from score -1.380089
[LightGBM] [Info] Start training from score -0.480790


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


🏃 View run LightGBM_TF-IDF Vectorizer at: https://dagshub.com/farjhanahmed/Demo-Exp.mlflow/#/experiments/1/runs/a9bd2990861147d086a0f73b1ef93e05
🧪 View experiment at: https://dagshub.com/farjhanahmed/Demo-Exp.mlflow/#/experiments/1
{'model_name': 'CatBoost', 'vectorizer_name': 'Count Vectorizer'}
Learning rate set to 0.090624
0:	learn: 1.0582204	total: 187ms	remaining: 3m 7s
1:	learn: 1.0238674	total: 295ms	remaining: 2m 27s
2:	learn: 0.9918690	total: 408ms	remaining: 2m 15s
3:	learn: 0.9659273	total: 531ms	remaining: 2m 12s
4:	learn: 0.9487522	total: 619ms	remaining: 2m 3s
5:	learn: 0.9312756	total: 712ms	remaining: 1m 57s
6:	learn: 0.9169998	total: 808ms	remaining: 1m 54s
7:	learn: 0.9047864	total: 896ms	remaining: 1m 51s
8:	learn: 0.8926769	total: 993ms	remaining: 1m 49s
9:	learn: 0.8826481	total: 1.09s	remaining: 1m 47s
10:	learn: 0.8738270	total: 1.18s	remaining: 1m 46s
11:	learn: 0.8638866	total: 1.28s	remaining: 1m 45s
12:	learn: 0.8572573	total: 1.37s	remaining: 1m 44s
13:	lear

In [ ]:
result

,model_name,vectorizer_name,accuracy
9,Logistic Regression,TF-IDF Vectorizer,0.758042
13,SGD Classifier,TF-IDF Vectorizer,0.756923
0,LightGBM,Count Vectorizer,0.753846
1,LightGBM,TF-IDF Vectorizer,0.749091
3,CatBoost,TF-IDF Vectorizer,0.747413
34,HistGradient Boosting,Count Vectorizer,0.745175
11,Ridge Classifier,TF-IDF Vectorizer,0.743217
35,HistGradient Boosting,TF-IDF Vectorizer,0.740979
12,SGD Classifier,Count Vectorizer,0.739301
8,Logistic Regression,Count Vectorizer,0.738182


In [ ]:
logistic = LogisticRegression()
sgd = SGDClassifier()
lgbm = LGBMClassifier()

emsomble = {
    "Voting Classifier": VotingClassifier(estimators=[("logistic", logistic),("sgd", sgd), ("lgbm", lgbm)], voting="hard"),
    "Stacking Classifier": StackingClassifier(estimators=[("logistic", logistic),("sgd", sgd), ("lgbm", lgbm)], final_estimator=LogisticRegression())
}

In [ ]:
dagshub.init(repo_owner='farjhanahmed', repo_name='Demo-Exp', mlflow=True)
mlflow.set_tracking_uri('https://dagshub.com/farjhanahmed/Demo-Exp.mlflow')
mlflow.set_experiment('Demo-Exp')

for model_name, model in emsomble.items():
  run_name = f"{model_name}"
  with mlflow.start_run(run_name=run_name):
    model.fit(X_train_vectorized, y_train)
    y_pred = model.predict(X_test_vectorized)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='micro')
    recall = recall_score(y_test, y_pred, average='micro')
    f1 = f1_score(y_test, y_pred, average='micro')
    cm = confusion_matrix(y_test, y_pred)[0][0]
    mlflow.log_param("model_name", model_name)
    mlflow.log_metrics({
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix_value": cm
    })


Initialized MLflow to track repo "farjhanahmed/Demo-Exp"

Repository farjhanahmed/Demo-Exp initialized!

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.309803 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 53414
[LightGBM] [Info] Number of data points in the train set: 14299, number of used features: 1763
[LightGBM] [Info] Start training from score -2.039076
[LightGBM] [Info] Start training from score -1.380089
[LightGBM] [Info] Start training from score -0.480790


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.204952 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 41979
[LightGBM] [Info] Number of data points in the train set: 11439, number of used features: 1454
[LightGBM] [Info] Start training from score -2.039596
[LightGBM] [Info] Start training from score -1.379933
[LightGBM] [Info] Start training from score -0.480744


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.115828 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 42214
[LightGBM] [Info] Number of data points in the train set: 11440, number of used features: 1464
[LightGBM] [Info] Start training from score -2.039011
[LightGBM] [Info] Start training from score -1.380020
[LightGBM] [Info] Start training from score -0.480831


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


🏃 View run Stacking Classifier at: https://dagshub.com/farjhanahmed/Demo-Exp.mlflow/#/experiments/1/runs/9b519bc3990e4dc09358878d516bcaa5
🧪 View experiment at: https://dagshub.com/farjhanahmed/Demo-Exp.mlflow/#/experiments/1
